In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set();

## Задаем все необходимые пути

In [2]:
PROJECT_FOLDER: str = 'P:\\Python Projects\\EDA_cardio'
DATASET_NAME: str = 'cardio_train.csv'
DATA_PATH: str = os.path.join(PROJECT_FOLDER, 'src', 'raw', DATASET_NAME)

RESULT_PATH: str = os.path.join(PROJECT_FOLDER, 'src', 'visualizations')

## Читаем и смотрим что внутри

In [3]:
data = pd.read_csv(os.path.join(DATA_PATH), sep=';')

In [4]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
id,70000.0,49972.419900,28851.302323,0.0,25006.75,50001.5,74889.25,99999.0
age,70000.0,19468.865814,2467.251667,10798.0,17664.00,19703.0,21327.00,23713.0
gender,70000.0,1.349571,0.476838,1.0,1.00,1.0,2.00,2.0
height,70000.0,164.359229,8.210126,55.0,159.00,165.0,170.00,250.0
weight,70000.0,74.205690,14.395757,10.0,65.00,72.0,82.00,200.0
ap_hi,70000.0,128.817286,154.011419,-150.0,120.00,120.0,140.00,16020.0
ap_lo,70000.0,96.630414,188.472530,-70.0,80.00,80.0,90.00,11000.0
cholesterol,70000.0,1.366871,0.680250,1.0,1.00,1.0,2.00,3.0
gluc,70000.0,1.226457,0.572270,1.0,1.00,1.0,1.00,3.0
smoke,70000.0,0.088129,0.283484,0.0,0.00,0.0,0.00,1.0


# Сводные таблицы

Для исследования трёх и более признаков полезным инструментов являются сводные таблицы (pivot tables). Этот инструмент хорошо знаком продвинутым пользователям электронных таблиц Excel, Google Spreadsheets. Рассмотрим, как с помощью сводной таблицы ответить на вопросы:

1) верно ли, что с возрастом люди становятся более склонны к употреблению алкоголя;
2) верно ли, что среди курящих процент ССЗ больше.

In [6]:
# values - признаки, по которым вычисляются значения функции aggfunc
# index - признаки, по которым выполняется группировка
data.pivot_table(values=['age', 'cardio'], index=['smoke', 'alco'], aggfunc='mean')

age    cardio
smoke alco                        
0     0     19508.899743  0.501462
      1     19391.964921  0.523037
1     0     19162.716107  0.487833
      1     18923.545307  0.444444

In [7]:
pd.crosstab(data['smoke'], data['alco'])

alco,0,1
smoke,,
0,61921,1910
1,4315,1854


# Выборка данных по условию. Способы индексирования в Pandas

In [8]:
h = data['height'] # сохраним всю колонку "рост" в отдельную переменную для экспериментов
type(h) # посмотрим тип 

pandas.core.series.Series

In [10]:
first_patient = data.iloc[0]
print(first_patient)

id                 0.0
age            18393.0
gender             2.0
height           168.0
weight            62.0
ap_hi            110.0
ap_lo             80.0
cholesterol        1.0
gluc               1.0
smoke              0.0
alco               0.0
active             1.0
cardio             0.0
Name: 0, dtype: float64


In [11]:
print(data.loc[0, 'age'])

18393


In [12]:
h_meters = h / 100 # предельно просто!
h_meters[:10] # в отдельных столбцах уже можно применять "обычные" срезы, как в списках

0    1.68
1    1.56
2    1.65
3    1.69
4    1.56
5    1.51
6    1.57
7    1.78
8    1.58
9    1.64
Name: height, dtype: float64

In [13]:
%%timeit
lilliputs = 0
for value in h:
    if value < 125:
        lilliputs = lilliputs + 1

5.32 ms ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit
h[h < 125].shape[0]

135 µs ± 276 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Итак, второй способ оказался быстрее приблизительно в 5 раз на наборе данных из 70000 значений (относительно небольшом). С ростом длины вектора циклы становятся в сотни и тысячи раз медленнее, чем векторизованные операции NumPy

In [16]:
# Вычислим средний возраст людей, склонных к курению
data[data['smoke'] == 1]['age'].mean() / 365

52.30366281251596